# Accessing OpenBIDS for CML Data
This tutorial shows how to load behavioral and EEG data in OpenBIDS format so it matches the structure used by the Computational Memory Lab’s CMLReader package. The structure is designed to support new analyses and replicate existing scripts with minimal changes.

In [74]:
# imports
import pandas as pd; pd.set_option('display.max_columns', None)
import cmlreaders as cml
import numpy as np
import xarray as xr
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals
import os
import mne
from ptsa.data.timeseries import TimeSeries

## Loading Behavioral Data

### CMLReader

As a reminder, to load behavioral data for a given experimental session, we select the session using its subject, experiment, and session identifiers, instantiate a CMLReader object, and then load the desired data type (e.g., events). 

In [75]:
# load dataframe of all sessions
df = cml.get_data_index()
df[:10]                     # show the first 10 entries

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/ltp/subjects/LTP001/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,0,NaN,NaN,0,LTP001,LTP001,NaN,protocols/ltp/subjects/LTP001/experiments/Valu...
1,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,0,NaN,NaN,0,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
2,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,1,NaN,NaN,1,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
3,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,10,NaN,NaN,10,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
4,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,11,NaN,NaN,11,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
5,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,12,NaN,NaN,12,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
6,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,13,NaN,NaN,13,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
7,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,14,NaN,NaN,14,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
8,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,15,NaN,NaN,15,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
9,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,16,NaN,NaN,16,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...


In [76]:
# let's find subjects who did the ValueCourier experiment
VC_df = df.query("experiment == 'ValueCourier'")
VC_df['subject'].unique()[:20]

array(['LTP001', 'LTP606', 'LTP607', 'LTP609', 'LTP610', 'LTP612',
       'LTP613', 'LTP614', 'LTP9992', 'LTP9993'], dtype=object)

In [77]:
# we'll pick LTP606 and select out this subject's ValueCourier sessions
sub_scalp = 'LTP606'
exp_scalp = 'ValueCourier'
df_select = df[(df['subject'] == sub_scalp) & (df['experiment'] == exp_scalp)]
display(df_select); print(f'{sub_scalp} sessions: {np.array(df_select.session)}')

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
7696,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,0,NaN,NaN,0,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7697,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,1,NaN,NaN,1,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7698,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,2,NaN,NaN,2,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7699,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,3,NaN,NaN,3,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7700,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,4,NaN,NaN,4,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7701,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,5,NaN,NaN,5,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...


LTP606 sessions: [0 1 2 3 4 5]


In [78]:
# lets load the data from the first session
df_sess = df_select.iloc[0]         # select 1 row

# instantiate a Reader object using session metadata
# subjects beginning with 'LTP' are scalp subjects, so we don't need to specify the localization and montage
cmlreader = cml.CMLReader(subject=df_sess['subject'], experiment=df_sess['experiment'], session=df_sess['session'])
# load the behavioral events
evs_cml = cmlreader.load('events')
evs_cml[:10]

,eegoffset,actualvalue,compensation,correctPointingDirection,eegfile,eogArtifact,experiment,finalrecalled,intruded,intrusion,item,itemno,itemvalue,montage,msoffset,mstime,multiplier,numingroupchosen,phase,playerrotY,presX,presZ,primacybuf,protocol,recalled,recencybuf,rectime,serialpos,session,store,storeX,storeZ,storepointtype,subject,submittedPointingDirection,trial,type,valuerecall
0,23440,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761665861795,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,store mappings,-999
1,31922,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-1,-999,0,-1,1761665865937,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,VIDEO_START,-999
2,468068,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-1,-999,0,-1,1761666078916,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,VIDEO_STOP,-999
3,646089,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666165847,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
4,646183,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666165893,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,TL_START,-999
5,1724585,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666692498,0.846154,4,1,-999.0,-3.000000,8.757812,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,TL_END,-999
6,1725717,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666693051,0.846154,4,1,-999.0,-3.937500,11.757812,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
7,1731210,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666695733,0.846154,4,1,-999.0,-3.953125,11.804688,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
8,1763086,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666711299,0.846154,4,1,-999.0,-3.953125,11.804688,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
9,1766295,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666712866,0.846154,4,1,-999.0,-3.953125,11.804688,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999


### OpenBIDS

To load behavioral data in OpenBIDS, we first set the root of the bids database. The database is structured as such:

<pre>
root/
├── subjects/
│   └── sub-{subject_id}/
│       └── ses-{session_id}/
│           ├── beh/
│           │   ├── sub-{subject_id}_ses-{session_id}_task-{experiment}_beh.json
│           │   └── sub-{subject_id}_ses-{session_id}_task-{experiment}_beh.tsv
│           └── eeg/
│               ├── sub-{subject_id}_ses-{session_id}_task-{experiment}_eeg.bdf
│               └── sub-{subject_id}_ses-{session_id}_task-{experiment}_events.tsv
└── participants.tsv
</pre>

We can use the get_entity_vals functon from the mne_bids package to find all the subjects in the database. Once we have selected a subject, we can use the same function to find all sessions and experiments (called 'task' in OpenBIDS format) associated with that subject. Finally, we can find the .tsv file containing the behavioral data and load it into a Pandas Dataframe.

In [79]:
# set root
bids_root = "/home1/maint/LTP_BIDS/"

In [80]:
# let's find subjects loaded in the database
subjects = get_entity_vals(bids_root, "subject")
subject = subjects[0]
subjects

['LTP606', 'LTP607', 'LTP609', 'LTP610', 'LTP612', 'LTP613', 'LTP614']

In [81]:
# now get the root of the subject and find which experiments they have completed
subject_root = os.path.join(bids_root, f"sub-{subjects[0]}")
tasks = get_entity_vals(subject_root, "task")
task = tasks[0]
tasks

['valuecourier']

In [82]:
# we can also see how many sessions they have completed
sessions = get_entity_vals(subject_root, "session")
session = sessions[0]
sessions

['0', '1', '2', '3', '4', '5']

In [83]:
# plug in the subject, task, and session info into BIDSPath and have the datatype set to "beh" to get the path to the .tsv file
# all inputs to BIDSPath should be strings so convert using the str() function
# load it using read_csv
path_bids = BIDSPath(
                subject=subject,
                session=str(session),
                task=task,  
                datatype="beh", 
                extension=".tsv",
                root=bids_root
            )
evs_bids = pd.read_csv(path_bids.fpath, sep="\t")
evs_bids[:10]

,mstime,trial_type,stim_file,actualvalue,compensation,eegfile,eogArtifact,experiment,intruded,intrusion,item,itemno,itemvalue,montage,msoffset,multiplier,numingroupchosen,phase,playerrotY,presX,presZ,primacybuf,protocol,recalled,rectime,recencybuf,serialpos,session,store,storeX,storeZ,storepointtype,subject,trial,valuerecall
0,0,store mappings,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
1,4142,VIDEO_START,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,-1.0,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
2,217121,VIDEO_STOP,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,-1.0,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
3,304052,SESS_START,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
4,304098,TL_START,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
5,830703,TL_END,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.000000,8.757812,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
6,831256,SESS_START,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.937500,11.757812,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
7,833938,SESS_START,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.953125,11.804688,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
8,849504,SESS_START,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.953125,11.804688,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
9,851071,SESS_START,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.953125,11.804688,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN


In [84]:
# we can also set the datatype to "eeg" and load it this way
# by setting it to "eeg", we avoid having to remake a BIDSPath object for both the EEG and behavioral data
bids_path = BIDSPath(
    subject=subject,
    session=str(session),
    task=task,
    datatype="eeg",
    root=bids_root,
)

events_path = os.path.join(bids_path.directory, bids_path.basename + "_events.tsv")
evs_bids = pd.read_csv(events_path, sep="\t")
evs_bids[:10]

,onset,duration,trial_type,sample,stim_file,actualvalue,compensation,eegfile,eogArtifact,experiment,intruded,intrusion,item,itemno,itemvalue,montage,msoffset,multiplier,numingroupchosen,phase,playerrotY,presX,presZ,primacybuf,protocol,recalled,rectime,recencybuf,serialpos,session,store,storeX,storeZ,storepointtype,subject,trial,valuerecall
0,11.445312,NaN,store mappings,23440,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
1,15.586914,NaN,VIDEO_START,31922,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,-1.0,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
2,228.548828,NaN,VIDEO_STOP,468068,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,-1.0,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
3,315.473145,NaN,SESS_START,646089,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
4,315.519043,NaN,TL_START,646183,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
5,842.082520,NaN,TL_END,1724585,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.000000,8.757812,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
6,842.635254,NaN,SESS_START,1725717,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.937500,11.757812,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
7,845.317383,NaN,SESS_START,1731210,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.953125,11.804688,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
8,860.881836,NaN,SESS_START,1763086,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.953125,11.804688,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
9,862.448730,NaN,SESS_START,1766295,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.953125,11.804688,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN


## Differences Between CML and OpenBIDS Events

The event dataframes loaded via OpenBIDS and CMLReader are identical in content, with the only differences being a small number of additional or renamed columns in the OpenBIDS version that conform to OpenBIDS formatting standards.

In [85]:
evs_cml.columns

Index(['eegoffset', 'actualvalue', 'compensation', 'correctPointingDirection',
       'eegfile', 'eogArtifact', 'experiment', 'finalrecalled', 'intruded',
       'intrusion', 'item', 'itemno', 'itemvalue', 'montage', 'msoffset',
       'mstime', 'multiplier', 'numingroupchosen', 'phase', 'playerrotY',
       'presX', 'presZ', 'primacybuf', 'protocol', 'recalled', 'recencybuf',
       'rectime', 'serialpos', 'session', 'store', 'storeX', 'storeZ',
       'storepointtype', 'subject', 'submittedPointingDirection', 'trial',
       'type', 'valuerecall'],
      dtype='object')

In [86]:
evs_bids.columns

Index(['onset', 'duration', 'trial_type', 'sample', 'stim_file', 'actualvalue',
       'compensation', 'eegfile', 'eogArtifact', 'experiment', 'intruded',
       'intrusion', 'item', 'itemno', 'itemvalue', 'montage', 'msoffset',
       'multiplier', 'numingroupchosen', 'phase', 'playerrotY', 'presX',
       'presZ', 'primacybuf', 'protocol', 'recalled', 'rectime', 'recencybuf',
       'serialpos', 'session', 'store', 'storeX', 'storeZ', 'storepointtype',
       'subject', 'trial', 'valuerecall'],
      dtype='object')

### Column differences
| CMLReader column | OpenBIDS column | Notes |
|------------------|-----------------|-------|
| `eegoffset` | `sample` | Same information; renamed to match BIDS conventions |
| `type` | `trial_type` | Renamed for BIDS compliance |
| `mstime` | `mstime` | In BIDS, `mstime` is typically shifted so that `mstime[0] = 0` |
| *(derived)* | `onset` | Computed as `sample / sample_frequency` |
| *(not present)* | `duration` | Not used / not applicable |
| *(not present)* | `stim_file` | Empty string placeholder required by BIDS |
                          

## Loading Raw EEG

Here's how we load the whole raw EEG file without creating epochs.

### CMLReader

In [87]:
# use reader from events
eeg_cml = cmlreader.load_eeg().to_ptsa()

Extracting EDF parameters from /protocols/ltp/subjects/LTP606/experiments/ValueCourier/sessions/0/ephys/current_processed/LTP606_session_0.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 10821631  =      0.000 ...  5284.000 secs...


### OpenBIDS

In [88]:
# use BIDSPath with datatype set to "eeg"
bids_path = BIDSPath(
            subject=subject,
            session=str(session),
            task=task,
            datatype="eeg",
            root=bids_root,
        )

# load the header from the bids file
raw = read_raw_bids(
    bids_path,
    verbose=True,
)

Extracting EDF parameters from /home1/maint/LTP_BIDS/sub-LTP606/ses-0/eeg/sub-LTP606_ses-0_task-valuecourier_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /home1/maint/LTP_BIDS/sub-LTP606/ses-0/eeg/sub-LTP606_ses-0_task-valuecourier_events.tsv.
Reading channel info from /home1/maint/LTP_BIDS/sub-LTP606/ses-0/eeg/sub-LTP606_ses-0_task-valuecourier_channels.tsv.
Reading electrode coords from /home1/maint/LTP_BIDS/sub-LTP606/ses-0/eeg/sub-LTP606_ses-0_space-CapTrak_electrodes.tsv.


/tmp/ipykernel_63073/1862152510.py:11: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw = read_raw_bids(
/tmp/ipykernel_63073/1862152510.py:11: RuntimeWarning: The unit for channel(s) Status has changed from NA to V.
  raw = read_raw_bids(
/tmp/ipykernel_63073/1862152510.py:11: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8']
  raw = read_raw_bids(
/tmp/ipykernel_63073/1862152510.py:11: RuntimeWarning: Not setting positions of 8 eog/misc channels found in montage:
['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = read_raw_bids(


In [89]:
# ptsa is just a wrapper for XArray, so we can just create an XArray object using the data from raw
eeg_bids = xr.DataArray(
    raw.get_data()[None, :, :],                # load the raw eeg data                  
    dims=("event", "channel", "time"),      
    coords={
        "event": [0],                          # singleton event index
        "channel": raw.ch_names,
        "time": raw.times  ,
        "samplerate": raw.info["sfreq"],          
    },
    name="eeg",
)
eeg_bids

<xarray.DataArray 'eeg' (event: 1, channel: 137, time: 10821632)>
array([[[ 6.90137787e-03,  6.89969038e-03,  6.90031537e-03, ...,
          6.13459804e-03,  6.13422304e-03,  6.13491054e-03],
        [ 1.91299803e-02,  1.91313553e-02,  1.91298240e-02, ...,
          1.37286778e-02,  1.37281778e-02,  1.37291778e-02],
        [ 1.89260588e-03,  1.89082463e-03,  1.88844963e-03, ...,
          2.63266701e-03,  2.63254201e-03,  2.63094826e-03],
        ...,
        [-2.59480724e-01, -2.59480661e-01, -2.59480880e-01, ...,
         -2.59476692e-01, -2.59476130e-01, -2.59476317e-01],
        [-2.59457130e-01, -2.59457786e-01, -2.59458161e-01, ...,
         -2.59454786e-01, -2.59454380e-01, -2.59453442e-01],
        [ 6.55360000e+04,  6.55360000e+04,  6.55360000e+04, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]])
Coordinates:
  * event       (event) int64 0
  * channel     (channel) <U6 'A1' 'A2' 'A3' 'A4' ... 'EXG7' 'EXG8' 'Status'
  * time        (time) float64 0.0 0.0004883 0.0009766 ... 5.284e+03 5.284e+03
    samplerate  float64 2.048e+03

## Loading Epoched EEG

Here's how to load Epoched EEG data. First let's set the constants.

In [90]:
# constants
REL_START, REL_STOP = 200, 3000
BUFFER_MS = 1000
WIDTH = 6

FREQS = np.logspace(np.log10(2), np.log10(100), 46)
NOTCH_BAND = (58., 62.)
BATCH_EVENTS = 64

### CMLReader

It is easy to load the eeg data using the load_eeg method of the Reader object.

In [92]:
# many of our analyses use the clean function, but to get the same data as the BIDS loader, we will forego it
# epochs_cml = reader.load_eeg(
#     word_evs_cml, rel_start=-BUFFER_MS, rel_stop=REL_STOP + BUFFER_MS, clean='LCF'
# ).to_ptsa()

# filter events
word_evs_cml = evs_cml[evs_cml["type"] == "WORD"]

epochs_cml = cmlreader.load_eeg(
    word_evs_cml, rel_start=-BUFFER_MS, rel_stop=REL_STOP + BUFFER_MS
).to_ptsa()
epochs_cml

Extracting EDF parameters from /protocols/ltp/subjects/LTP606/experiments/ValueCourier/sessions/0/ephys/current_processed/LTP606_session_0.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 10821631  =      0.000 ...  5284.000 secs...
Not setting metadata
75 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 75 events and 10241 original time points ...
0 bad epochs dropped


<xarray.TimeSeries (event: 75, channel: 137, time: 10241)>
array([[[ 5.16056859e-03,  5.16216234e-03,  5.16313108e-03, ...,
          5.12997490e-03,  5.12759990e-03,  5.12894365e-03],
        [ 1.72542962e-02,  1.72552962e-02,  1.72559837e-02, ...,
          1.72135463e-02,  1.72126088e-02,  1.72149838e-02],
        [ 2.91476024e-03,  2.91541649e-03,  2.91554149e-03, ...,
          2.91991648e-03,  2.92169773e-03,  2.92191648e-03],
        ...,
        [-2.59470442e-01, -2.59471567e-01, -2.59470474e-01, ...,
         -2.59471505e-01, -2.59469849e-01, -2.59471630e-01],
        [-2.59448005e-01, -2.59448442e-01, -2.59448661e-01, ...,
         -2.59447630e-01, -2.59447974e-01, -2.59449130e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 5.11297493e-03,  5.11309993e-03,  5.11391242e-03, ...,
          5.15184985e-03,  5.15050611e-03,  5.15172485e-03],
        [ 1.71282652e-02,  1.71294215e-02,  1.71288277e-02, ...,
          1.71810776e-02,  1.71810776e-02,  1.71796089e-02],
        [ 2.90066651e-03,  2.89904152e-03,  2.89938527e-03, ...,
          2.91732273e-03,  2.91938523e-03,  2.91960398e-03],
...
        [-2.59476192e-01, -2.59475192e-01, -2.59476192e-01, ...,
         -2.59476536e-01, -2.59476630e-01, -2.59476599e-01],
        [-2.59453536e-01, -2.59453755e-01, -2.59454442e-01, ...,
         -2.59453192e-01, -2.59455130e-01, -2.59454786e-01],
        [ 8.00000000e+00,  8.00000000e+00,  8.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 6.12800430e-03,  6.12797305e-03,  6.12778555e-03, ...,
          6.06759816e-03,  6.06575442e-03,  6.06603567e-03],
        [ 1.40125210e-02,  1.40118647e-02,  1.40107085e-02, ...,
          1.39741148e-02,  1.39731148e-02,  1.39754273e-02],
        [ 2.66879194e-03,  2.66504195e-03,  2.66697945e-03, ...,
          2.63738575e-03,  2.63657325e-03,  2.63622950e-03],
        ...,
        [-2.59477692e-01, -2.59478067e-01, -2.59476255e-01, ...,
         -2.59475786e-01, -2.59476411e-01, -2.59478161e-01],
        [-2.59454786e-01, -2.59454599e-01, -2.59454349e-01, ...,
         -2.59453786e-01, -2.59453317e-01, -2.59454067e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          8.00000000e+00,  8.00000000e+00,  8.00000000e+00]]])
Coordinates: (12/42)
  * event                       (event) object MultiIndex
  * actualvalue                 (event) float64 14.07 14.07 ... 34.93 34.93
  * compensation                (event) float64 8.462 8.462 ... 8.462 8.462
  * correctPointingDirection    (event) int64 -999 -999 -999 ... -999 -999 -999
  * eegfile                     (event) object '/protocols/ltp/subjects/LTP60...
  * eegoffset                   (event) int64 4708671 4841846 ... 10541679
    ...                          ...
  * trial                       (event) int64 0 0 0 0 0 0 0 0 ... 4 4 4 4 4 4 4
  * type                        (event) object 'WORD' 'WORD' ... 'WORD' 'WORD'
  * valuerecall                 (event) int64 15 15 15 15 15 ... 22 22 22 22 22
  * channel                     (channel) <U6 'A1' 'A2' 'A3' ... 'EXG8' 'Status'
  * time                        (time) float64 -1e+03 -999.5 ... 4e+03 4e+03
    samplerate                  float64 2.048e+03

### OpenBIDS

We can load epochs for OpenBIDS using the MNE Epochs object then converting to ptsa.

REMINDER: MNE Epochs uses seconds so we must divide by 1000 if the constants were in seconds.

In [93]:
# get BIDSPath
bids_path = BIDSPath(
    subject=subject,
    session=str(session),
    task=task,
    datatype="eeg",
    root=bids_root,
)

# and load raw data
raw = read_raw_bids(bids_path)

# set non-EEG channels
raw.set_channel_types({
    "EXG1": "eog", "EXG2": "eog", "EXG3": "eog", "EXG4": "eog",
    "EXG5": "misc", "EXG6": "misc", "EXG7": "misc", "EXG8": "misc",
})

Extracting EDF parameters from /home1/maint/LTP_BIDS/sub-LTP606/ses-0/eeg/sub-LTP606_ses-0_task-valuecourier_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /home1/maint/LTP_BIDS/sub-LTP606/ses-0/eeg/sub-LTP606_ses-0_task-valuecourier_events.tsv.
Reading channel info from /home1/maint/LTP_BIDS/sub-LTP606/ses-0/eeg/sub-LTP606_ses-0_task-valuecourier_channels.tsv.
Reading electrode coords from /home1/maint/LTP_BIDS/sub-LTP606/ses-0/eeg/sub-LTP606_ses-0_space-CapTrak_electrodes.tsv.


/tmp/ipykernel_63073/588611710.py:11: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw = read_raw_bids(bids_path)
/tmp/ipykernel_63073/588611710.py:11: RuntimeWarning: The unit for channel(s) Status has changed from NA to V.
  raw = read_raw_bids(bids_path)
/tmp/ipykernel_63073/588611710.py:11: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8']
  raw = read_raw_bids(bids_path)
/tmp/ipykernel_63073/588611710.py:11: RuntimeWarning: Not setting positions of 8 eog/misc channels found in montage:
['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = read_raw_bids(bids_path)


<RawEDF | sub-LTP606_ses-0_task-valuecourier_eeg.bdf, 137 x 10821632 (5284.0 s), ~154 kB, data not loaded>

### Filtering Events for Loading

In order to select the events that we will epoch over, we must get the string representation from the events from the raw data's header using the events_from_annotations function. The function returns "events" which maps the sample to the event index and "event_id" which maps each event name to its index.

In [94]:
# get events from raw data's header
events, event_id = mne.events_from_annotations(raw)

Used Annotations descriptions: ['FINAL_COMPENSATION', 'POINTER_ON', 'PRACTICE_DELIVERY_END', 'PRACTICE_DELIVERY_START', 'PRACTICE_VALUE_RECALL', 'PRACTICE_WORD', 'REC_START', 'REC_STOP', 'REC_WORD', 'REC_WORD_VV', 'SESS_END', 'SESS_START', 'TL_END', 'TL_START', 'TRIAL_END', 'TRIAL_START', 'VALUE_RECALL', 'VIDEO_START', 'VIDEO_STOP', 'WORD', 'store mappings']


To filter the events, we select only the event_ids we are interested in.

In [95]:
# filter word events and return the index of the WORD event
word_event_id = {k: v for k, v in event_id.items() if k == "WORD"}
word_event_id

{'WORD': 20}

In [96]:
tmin = (-BUFFER_MS / 1000)
tmax = ((REL_STOP /1000 + BUFFER_MS / 1000))

# load into MNE Epochs object
epochs_mne = mne.Epochs(
    raw,
    events=events,                      # we can now load the events here
    event_id=word_event_id,             # we cna also load the filtered events here
    tmin=tmin,
    tmax=tmax,
    baseline=(None, 0), # None goes to start, 0 is onset of stimuli
    preload=True,
    event_repeated="merge",
)
epochs_mne

Not setting metadata
75 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 75 events and 10241 original time points ...
0 bad epochs dropped


Number of events,75
Events,WORD: 75
Time range,-1.000 – 4.000 s
Baseline,-1.000 – 0.000 s


In [97]:
# convert to ptsa
epochs_bids = TimeSeries.from_mne_epochs(epochs_mne, word_evs_bids)
epochs_bids

<xarray.TimeSeries (event: 75, channel: 137, time: 10241)>
array([[[ 2.65767109e-05,  2.81704580e-05,  2.91392062e-05, ...,
         -4.01698257e-06, -6.39197818e-06, -5.04823067e-06],
        [ 1.13833215e-05,  1.23833196e-05,  1.30708183e-05, ...,
         -2.93666033e-05, -3.03041015e-05, -2.79291059e-05],
        [ 4.15029431e-06,  4.80654310e-06,  4.93154286e-06, ...,
          9.30653478e-06,  1.10877815e-05,  1.13065311e-05],
        ...,
        [-2.59470442e-01, -2.59471567e-01, -2.59470474e-01, ...,
         -2.59471505e-01, -2.59469849e-01, -2.59471630e-01],
        [-2.59448005e-01, -2.59448442e-01, -2.59448661e-01, ...,
         -2.59447630e-01, -2.59447974e-01, -2.59449130e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-1.61781210e-05, -1.60531213e-05, -1.52406228e-05, ...,
          2.26968071e-05,  2.13530596e-05,  2.25718074e-05],
        [-9.34016393e-06, -8.18391606e-06, -8.77766497e-06, ...,
          4.34722385e-05,  4.34722385e-05,  4.20034912e-05],
        [-2.19560966e-06, -3.82060665e-06, -3.47685729e-06, ...,
          1.44606096e-05,  1.65231058e-05,  1.67418554e-05],
...
        [-2.59476192e-01, -2.59475192e-01, -2.59476192e-01, ...,
         -2.59476536e-01, -2.59476630e-01, -2.59476599e-01],
        [-2.59453536e-01, -2.59453755e-01, -2.59454442e-01, ...,
         -2.59453192e-01, -2.59455130e-01, -2.59454786e-01],
        [ 8.00000000e+00,  8.00000000e+00,  8.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-1.13152548e-06, -1.16277543e-06, -1.35027508e-06, ...,
         -6.15376639e-05, -6.33814105e-05, -6.31001610e-05],
        [ 3.93677591e-06,  3.28052713e-06,  2.12427926e-06, ...,
         -3.44694031e-05, -3.54694013e-05, -3.31569055e-05],
        [ 5.60607798e-06,  1.85608491e-06,  3.79358133e-06, ...,
         -2.58001140e-05, -2.66126125e-05, -2.69563619e-05],
        ...,
        [-2.59477692e-01, -2.59478067e-01, -2.59476255e-01, ...,
         -2.59475786e-01, -2.59476411e-01, -2.59478161e-01],
        [-2.59454786e-01, -2.59454599e-01, -2.59454349e-01, ...,
         -2.59453786e-01, -2.59453317e-01, -2.59454067e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          8.00000000e+00,  8.00000000e+00,  8.00000000e+00]]])
Coordinates: (12/41)
  * event             (event) object MultiIndex
  * onset             (event) float64 2.299e+03 2.364e+03 ... 5.147e+03
  * duration          (event) float64 nan nan nan nan nan ... nan nan nan nan
  * trial_type        (event) object 'WORD' 'WORD' 'WORD' ... 'WORD' 'WORD'
  * sample            (event) int64 4708671 4841846 ... 10449666 10541679
  * stim_file         (event) object 'wordpools/valuecourier_wordpool.txt' .....
    ...                ...
  * subject           (event) object 'LTP606' 'LTP606' ... 'LTP606' 'LTP606'
  * trial             (event) int64 0 0 0 0 0 0 0 0 0 0 ... 4 4 4 4 4 4 4 4 4 4
  * valuerecall       (event) float64 15.0 15.0 15.0 15.0 ... 22.0 22.0 22.0
  * channel           (channel) <U6 'A1' 'A2' 'A3' ... 'EXG7' 'EXG8' 'Status'
  * time              (time) float64 -1.0 -0.9995 -0.999 ... 3.999 4.0 4.0
    samplerate        float64 2.048e+03